In [ ]:
pip install lime shap cudaq

In [ ]:
!pip install matplotlib==3.8.4
!pip install torch==2.2.2
!pip install torchvision==0.17.0
!pip install scikit-learn==1.4.2

# **Novel Adversiral algorithm with GANs**

In [ ]:
import cudaq
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Function
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from lime.lime_tabular import LimeTabularExplainer

In [ ]:
device = torch.device("cpu")
cudaq.set_target("qpp-cpu")

In [ ]:
class Generator(nn.Module):
    def __init__(self, latent_dim=16, feature_dim=8):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(latent_dim, 64),
            nn.ReLU(),
            nn.Linear(64, feature_dim),
        )

    def forward(self, z):
        # z shape: (batch_size, latent_dim)
        return self.net(z)  # shape: (batch_size, feature_dim)

class Discriminator(nn.Module):
    def __init__(self, feature_dim=8):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(feature_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        # x shape: (batch_size, feature_dim)
        return self.net(x).view(-1)  # shape: (batch_size,)

def train_gan(generator, discriminator, data_real, gan_epochs=100, latent_dim=16):
    optim_g = optim.Adam(generator.parameters(), lr=0.0002)
    optim_d = optim.Adam(discriminator.parameters(), lr=0.0002)
    criterion = nn.BCELoss()

    for epoch in range(gan_epochs):
        # ----------- Discriminator Update -----------
        discriminator.train()
        generator.train()

        # 1) Real data
        real_labels = torch.ones(data_real.size(0), device=data_real.device)
        preds_real = discriminator(data_real)
        loss_d_real = criterion(preds_real, real_labels)

        # 2) Fake data
        z = torch.randn(data_real.size(0), latent_dim, device=data_real.device)
        fake_data = generator(z)
        fake_labels = torch.zeros(data_real.size(0), device=data_real.device)
        preds_fake = discriminator(fake_data.detach())
        loss_d_fake = criterion(preds_fake, fake_labels)

        # Combine
        loss_d = loss_d_real + loss_d_fake
        optim_d.zero_grad()
        loss_d.backward()
        optim_d.step()
        preds_fake_for_g = discriminator(fake_data)
        loss_g = criterion(preds_fake_for_g, real_labels)

        optim_g.zero_grad()
        loss_g.backward()
        optim_g.step()

        if (epoch + 1) % 20 == 0:
            print(f"[GAN] Epoch {epoch+1}/{gan_epochs} | "
                  f"D Loss: {loss_d.item():.4f} | G Loss: {loss_g.item():.4f}")
    return generator, discriminator

In [ ]:
def ry(theta, qubit):
    cudaq.ry(theta, qubit)

def rx(theta, qubit):
    cudaq.rx(theta, qubit)

class QuantumFunction(Function):
    def __init__(self, qubit_count: int, hamiltonian: cudaq.SpinOperator):
        @cudaq.kernel
        def kernel(qubit_count: int, thetas: np.ndarray):
            qubits = cudaq.qvector(qubit_count)
            ry(thetas[0], qubits[0])
            rx(thetas[1], qubits[0])

        self.kernel = kernel
        self.qubit_count = qubit_count
        self.hamiltonian = hamiltonian

    def run(self, theta_vals: torch.Tensor) -> torch.Tensor:
        theta_vals_np = theta_vals.cpu().numpy()
        qubit_counts = [self.qubit_count] * theta_vals_np.shape[0]
        results = cudaq.observe(self.kernel, self.hamiltonian, qubit_counts, theta_vals_np)
        exp_vals = [r.expectation() for r in results]
        return torch.tensor(exp_vals, dtype=torch.float32, device=device)

    @staticmethod
    def forward(ctx, thetas: torch.Tensor, quantum_circuit, shift) -> torch.Tensor:
        ctx.shift = shift
        ctx.quantum_circuit = quantum_circuit
        exp_vals = ctx.quantum_circuit.run(thetas).view(-1, 1)
        ctx.save_for_backward(thetas, exp_vals)
        return exp_vals

    @staticmethod
    def backward(ctx, grad_output):
        thetas, _ = ctx.saved_tensors
        gradients = torch.zeros_like(thetas)

        for i in range(thetas.shape[1]):
            thetas_plus = thetas.clone()
            thetas_plus[:, i] += ctx.shift
            exp_vals_plus = ctx.quantum_circuit.run(thetas_plus)

            thetas_minus = thetas.clone()
            thetas_minus[:, i] -= ctx.shift
            exp_vals_minus = ctx.quantum_circuit.run(thetas_minus)

            gradients[:, i] = (exp_vals_plus - exp_vals_minus) / (2.0 * ctx.shift)

        return gradients * grad_output, None, None

class QuantumLayer(nn.Module):
    def __init__(self, qubit_count: int, hamiltonian, shift: float):
        super().__init__()
        self.quantum_circuit = QuantumFunction(qubit_count, hamiltonian)
        self.shift = shift

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return QuantumFunction.apply(x, self.quantum_circuit, self.shift)

In [ ]:
class Hybrid_QNN(nn.Module):
    """
    Model 1: A feedforward net that routes 2 outputs to a quantum layer.
    """
    def __init__(self, input_dim=8):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.fc2 = nn.Linear(256, 128)
        self.dropout1 = nn.Dropout(0.25)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 32)
        self.fc5 = nn.Linear(32, 2)  # 2 parameters for quantum circuit
        self.dropout2 = nn.Dropout(0.25)

        self.quantum = QuantumLayer(
            qubit_count=2,
            hamiltonian=cudaq.spin.z(0),
            shift=np.pi / 2
        )

    def forward(self, x: torch.Tensor):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.dropout1(x)
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = torch.relu(self.fc5(x))
        x = self.dropout2(x)  # shape: (batch, 2)
        out = torch.sigmoid(self.quantum(x)).view(-1)  # shape: (batch,)
        return out

class EvaluatorModel(nn.Module):
    def __init__(self, input_dim=17):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x).squeeze(-1)

In [ ]:
def lime_explanations(model, lime_explainer, X_batch_np, feature_count):
    explanations = []
    for row in X_batch_np:
        exp = lime_explainer.explain_instance(
            data_row=row,
            predict_fn=lambda z: (
                model(
                    torch.tensor(z, dtype=torch.float32, device=device)
                ).cpu().detach().numpy()
            ),
            num_features=feature_count
        )
        exp_map = exp.as_map()
        label_key = next(iter(exp_map))
        local_pairs = exp_map[label_key]
        row_expl = np.zeros(feature_count)
        for feat_idx, weight in local_pairs:
            row_expl[feat_idx] = weight
        explanations.append(row_expl)
    return np.array(explanations)

In [ ]:
dataset_path = "/content/tablea1.csv"
df = pd.read_csv(dataset_path)

numeric_cols = df.select_dtypes(include=[np.number]).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())

features = ['logM1/2', 'logRe', 'logAge', '[Z/H]', 'logM*/L', 'DlogAge', 'D[Z/H]', 'DlogM*/L']
target = 'logsigmae'

X_np = df[features].values  # shape (N, 8)
y_np = df[target].values.reshape(-1, 1)

scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_np = scaler_X.fit_transform(X_np)
y_np = scaler_y.fit_transform(y_np).flatten()

X = torch.tensor(X_np, dtype=torch.float32, device=device)
y = torch.tensor(y_np, dtype=torch.float32, device=device)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
latent_dim = 16
feature_dim = X_train.shape[1]

generator = Generator(latent_dim=latent_dim, feature_dim=feature_dim).to(device)
discriminator = Discriminator(feature_dim=feature_dim).to(device)

# Train the GAN on the real X_train only (unconditional, ignoring y)
print("\n--- Training GAN to Generate Synthetic X ---")
generator, discriminator = train_gan(
    generator, discriminator,
    data_real=X_train,  # shape (N,8)
    gan_epochs=100,
    latent_dim=latent_dim
)

# Generate synthetic data (size e.g. N/2 or same N)
num_syn = X_train.shape[0] // 2
z = torch.randn(num_syn, latent_dim, device=device)
X_syn = generator(z).detach()  # shape (num_syn, 8)
X_train_final = torch.cat([X_train, X_syn], dim=0)
idxs = torch.randint(0, y_train.shape[0], size=(num_syn,))
y_syn = y_train[idxs]
y_train_final = torch.cat([y_train, y_syn], dim=0)


--- Training GAN to Generate Synthetic X ---
[GAN] Epoch 20/100 | D Loss: 1.5098 | G Loss: 0.7578
[GAN] Epoch 40/100 | D Loss: 1.4775 | G Loss: 0.7398
[GAN] Epoch 60/100 | D Loss: 1.4444 | G Loss: 0.7277
[GAN] Epoch 80/100 | D Loss: 1.4132 | G Loss: 0.7174
[GAN] Epoch 100/100 | D Loss: 1.3863 | G Loss: 0.7072


In [ ]:
X_train_np_final = X_train_final.cpu().numpy()

lime_explainer = LimeTabularExplainer(
    training_data=X_train_np_final,
    feature_names=features,
    discretize_continuous=True,
    mode='regression'
)

model1 = Hybrid_QNN(input_dim=feature_dim).to(device)
model2 = EvaluatorModel(input_dim=(feature_dim + 1 + feature_dim)).to(device)

optimizer1 = optim.Adam(model1.parameters(), lr=0.001)
optimizer2 = optim.Adam(model2.parameters(), lr=0.001)
mse_loss = nn.MSELoss()

alpha = 0.5
epochs = 2

print("\n--- Training Model 1 (QNN) + Model 2 (Evaluator) with Synthetic + Real Data ---")
for epoch in range(epochs):
    model1.train()
    model2.train()

    # -----------------------------------------------------
    # (A) Forward pass for Model 1 on combined real+syn data
    # -----------------------------------------------------
    y_hat = model1(X_train_final)
    loss_m1_mse = mse_loss(y_hat, y_train_final)

    # -----------------------------------------------------
    # (B) LIME explanations on entire (real+syn) training set
    # -----------------------------------------------------
    lime_expls = lime_explanations(
        model=model1,
        lime_explainer=lime_explainer,
        X_batch_np=X_train_np_final,
        feature_count=feature_dim
    )

    # -----------------------------------------------------
    # (C) Prepare input for Model 2 => [X, y_hat, LIME]
    # -----------------------------------------------------
    y_hat_np = y_hat.detach().cpu().numpy().reshape(-1, 1)
    input_model2_np = np.concatenate([X_train_np_final, y_hat_np, lime_expls], axis=1)
    input_model2 = torch.tensor(input_model2_np, dtype=torch.float32, device=device)

    # -----------------------------------------------------
    # (D) Forward pass for Model 2
    # -----------------------------------------------------
    y_eval_pred = model2(input_model2)
    loss_m2 = mse_loss(y_eval_pred, y_train_final)

    # -----------------------------------------------------
    # (E) Feedback loss for Model 1
    # -----------------------------------------------------
    feedback_loss = mse_loss(y_eval_pred, y_train_final)
    loss_m1_total = loss_m1_mse + alpha * feedback_loss

    # -----------------------------------------------------
    # (F) Backprop & Optimization
    # -----------------------------------------------------
    optimizer1.zero_grad()
    optimizer2.zero_grad()

    loss_m1_total.backward(retain_graph=True)
    optimizer2.zero_grad()
    loss_m2.backward()

    optimizer1.step()
    optimizer2.step()

    print(f"Epoch [{epoch+1}/{epochs}] | "
          f"M1 MSE: {loss_m1_mse.item():.4f} | "
          f"M2 MSE: {loss_m2.item():.4f} | "
          f"Total M1 Loss: {loss_m1_total.item():.4f}")


--- Training Model 1 (QNN) + Model 2 (Evaluator) with Synthetic + Real Data ---
Epoch [1/2] | M1 MSE: 0.0789 | M2 MSE: 0.6483 | Total M1 Loss: 0.4030
Epoch [2/2] | M1 MSE: 0.0789 | M2 MSE: 0.6369 | Total M1 Loss: 0.3973


In [ ]:
model1.eval()
with torch.no_grad():
    y_hat_test = model1(X_test)
    test_mse = mse_loss(y_hat_test, y_test).item()
    test_rmse = np.sqrt(test_mse)
    test_mae = mean_absolute_error(y_test.cpu().numpy(), y_hat_test.cpu().numpy())
    r2 = -r2_score(y_test.cpu().numpy(), y_hat_test.cpu().numpy())

print("\n--- Final Evaluation of Model 1 on O  riginal Test ---")
print(f"MSE:  {test_mse:.4f}")
print(f"RMSE: {test_rmse:.4f}")
print(f"MAE:  {test_mae:.4f}")
print(f"R^2:  {r2:.4f}")

# Accuracy by error
y_max = torch.max(y).item()
y_min = torch.min(y).item()
data_range = y_max - y_min

mae_accuracy = (1.0 - test_mae / data_range) * 100.0
rmse_accuracy = (1.0 - test_rmse / data_range) * 100.0
mse_accuracy = (1.0 - test_mse / data_range) * 100.0

print("\n--- Accuracy by Error ---")
print(f"MSE:  {test_mse:.4f},   Acc: {mse_accuracy:.2f}%")
print(f"RMSE: {test_rmse:.4f},  Acc: {rmse_accuracy:.2f}%")
print(f"MAE:  {test_mae:.4f},   Acc: {mae_accuracy:.2f}%")
print(f"R^2:  {r2:.4f}")


--- Final Evaluation of Model 1 on O  riginal Test ---
MSE:  0.0773
RMSE: 0.2781
MAE:  0.2219
R^2:  0.6451

--- Accuracy by Error ---
MSE:  0.0773,   Acc: 92.27%
RMSE: 0.2781,  Acc: 72.19%
MAE:  0.2219,   Acc: 77.81%
R^2:  0.6451


# **Novel Adversiral Algorithm with 4 Models in the Adversiral Network**

In [ ]:
import cudaq
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Function
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from lime.lime_tabular import LimeTabularExplainer

In [ ]:
device = torch.device("cpu")
cudaq.set_target("qpp-cpu")

In [ ]:
df = pd.read_csv("/content/tablea1.csv")

numeric_cols = df.select_dtypes(include=[np.number]).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())

features = ['logM1/2', 'logRe', 'logAge', '[Z/H]', 'logM*/L', 'DlogAge', 'D[Z/H]', 'DlogM*/L']
target = 'logsigmae'

X_np = df[features].values
y_np = df[target].values.reshape(-1, 1)

# Scale features (8D) and target (1D)
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_np = scaler_X.fit_transform(X_np)
y_np = scaler_y.fit_transform(y_np).flatten()

# Convert to torch
X = torch.tensor(X_np, dtype=torch.float32, device=device)
y = torch.tensor(y_np, dtype=torch.float32, device=device)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [ ]:
class Generator(nn.Module):

    def __init__(self, z_dim=16, feature_dim=8):
        super().__init__()
        self.fc1 = nn.Linear(z_dim, 64)
        self.fc2 = nn.Linear(64, 128)
        self.fc3 = nn.Linear(128, feature_dim)

    def forward(self, z):
        x = torch.relu(self.fc1(z))
        x = torch.relu(self.fc2(x))
        # Synthetic features in [batch_size, feature_dim]
        return torch.sigmoid(self.fc3(x))

class Discriminator(nn.Module):

    def __init__(self, feature_dim=8):
        super().__init__()
        self.fc1 = nn.Linear(feature_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return torch.sigmoid(self.fc3(x))

In [ ]:
def ry(theta, qubit):
    cudaq.ry(theta, qubit)

def rx(theta, qubit):
    cudaq.rx(theta, qubit)

class QuantumFunction(Function):

    def __init__(self, qubit_count: int, hamiltonian: cudaq.SpinOperator):
        @cudaq.kernel
        def kernel(qubit_count: int, thetas: np.ndarray):
            qubits = cudaq.qvector(qubit_count)
            # Very simple circuit using two parameters on qubit 0
            ry(thetas[0], qubits[0])
            rx(thetas[1], qubits[0])

        self.kernel = kernel
        self.qubit_count = qubit_count
        self.hamiltonian = hamiltonian

    def run(self, theta_vals: torch.Tensor) -> torch.Tensor:
        theta_vals_np = theta_vals.cpu().numpy()
        qubit_counts = [self.qubit_count] * theta_vals_np.shape[0]
        # Evaluate <hamiltonian> for each sample
        results = cudaq.observe(self.kernel, self.hamiltonian, qubit_counts, theta_vals_np)
        exp_vals = [r.expectation() for r in results]
        return torch.tensor(exp_vals, dtype=torch.float32, device=device)

    @staticmethod
    def forward(ctx, thetas: torch.Tensor, quantum_circuit, shift) -> torch.Tensor:
        ctx.shift = shift
        ctx.quantum_circuit = quantum_circuit

        exp_vals = ctx.quantum_circuit.run(thetas).view(-1, 1)
        ctx.save_for_backward(thetas, exp_vals)
        return exp_vals

    @staticmethod
    def backward(ctx, grad_output):
        thetas, _ = ctx.saved_tensors
        gradients = torch.zeros_like(thetas)

        # Parameter-shift rule
        for i in range(thetas.shape[1]):
            thetas_plus = thetas.clone()
            thetas_plus[:, i] += ctx.shift
            exp_vals_plus = ctx.quantum_circuit.run(thetas_plus)

            thetas_minus = thetas.clone()
            thetas_minus[:, i] -= ctx.shift
            exp_vals_minus = ctx.quantum_circuit.run(thetas_minus)

            gradients[:, i] = (exp_vals_plus - exp_vals_minus) / (2.0 * ctx.shift)

        return gradients * grad_output, None, None

class QuantumLayer(nn.Module):
    def __init__(self, qubit_count: int, hamiltonian, shift: float):
        super().__init__()
        self.quantum_circuit = QuantumFunction(qubit_count, hamiltonian)
        self.shift = shift

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return QuantumFunction.apply(x, self.quantum_circuit, self.shift)

In [ ]:
class Hybrid_QNN(nn.Module):

    def __init__(self, input_dim=8):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.fc2 = nn.Linear(256, 128)
        self.dropout1 = nn.Dropout(0.25)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 32)
        self.fc5 = nn.Linear(32, 2)

        self.dropout2 = nn.Dropout(0.25)
        self.quantum = QuantumLayer(
            qubit_count=2,
            hamiltonian=cudaq.spin.z(0),
            shift=np.pi / 2
        )

    def forward(self, x: torch.Tensor):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.dropout1(x)
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = torch.relu(self.fc5(x))   # shape: (batch, 2)
        x = self.dropout2(x)
        # feed into quantum circuit => shape: (batch,)
        out = torch.sigmoid(self.quantum(x)).view(-1)
        return out

class EvaluatorModel(nn.Module):

    def __init__(self, input_dim=17):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x).squeeze(-1)

In [ ]:
def lime_explanations(model, lime_explainer, X_batch_np, feature_count):
    explanations = []
    for row in X_batch_np:
        exp = lime_explainer.explain_instance(
            data_row=row,
            predict_fn=lambda z: (
                model(
                    torch.tensor(z, dtype=torch.float32, device=device)
                ).cpu().detach().numpy()
            ),
            num_features=feature_count
        )
        exp_map = exp.as_map()
        label_key = next(iter(exp_map))
        local_pairs = exp_map[label_key]

        row_expl = np.zeros(feature_count)
        for feat_idx, weight in local_pairs:
            row_expl[feat_idx] = weight
        explanations.append(row_expl)
    return np.array(explanations)

In [ ]:
z_dim = 16            # dimension of random noise
feature_dim = 8       # dimension of features

generator = Generator(z_dim=z_dim, feature_dim=feature_dim).to(device)
discriminator = Discriminator(feature_dim=feature_dim).to(device)

model1 = Hybrid_QNN(input_dim=feature_dim).to(device)
model2 = EvaluatorModel(input_dim=feature_dim + 1 + feature_dim).to(device)

# Optimizers
lr = 0.001
opt_g = optim.Adam(generator.parameters(), lr=lr)
opt_d = optim.Adam(discriminator.parameters(), lr=lr)
opt_m1 = optim.Adam(model1.parameters(), lr=lr)
opt_m2 = optim.Adam(model2.parameters(), lr=lr)

mse_loss = nn.MSELoss()
bce_loss = nn.BCELoss()  # for GAN real/fake classification

# LIME explainer
X_train_np = X_train.cpu().numpy()
lime_explainer = LimeTabularExplainer(
    training_data=X_train_np,
    feature_names=features,
    discretize_continuous=True,
    mode='regression'
)

# Combined hyperparams
alpha = 0.5
epochs = 2
batch_size = 64

In [ ]:
n_train = X_train.size(0)

for epoch in range(epochs):
    # Shuffle real data indices for mini-batching
    perm = torch.randperm(n_train, device=device)

    # --------------
    # (A) TRAIN GAN
    # --------------
    generator.train()
    discriminator.train()

    # We'll do a few mini-batches for the GAN:
    gan_steps = n_train // batch_size
    for step in range(gan_steps):
        idx = perm[step*batch_size : (step+1)*batch_size]
        real_x = X_train[idx]  # shape: (batch_size, 8)

        # (i) Train discriminator
        opt_d.zero_grad()

        # real samples => label=1
        real_labels = torch.ones(real_x.size(0), 1, device=device)
        pred_real = discriminator(real_x)
        d_loss_real = bce_loss(pred_real, real_labels)

        # fake samples => label=0
        noise = torch.randn(real_x.size(0), z_dim, device=device)
        fake_x = generator(noise).detach()  # shape: (batch_size, 8)
        fake_labels = torch.zeros(fake_x.size(0), 1, device=device)
        pred_fake = discriminator(fake_x)
        d_loss_fake = bce_loss(pred_fake, fake_labels)

        d_loss = d_loss_real + d_loss_fake
        d_loss.backward()
        opt_d.step()

        # (ii) Train generator
        opt_g.zero_grad()
        noise = torch.randn(real_x.size(0), z_dim, device=device)
        gen_x = generator(noise)  # shape: (batch_size, 8)
        pred_gen = discriminator(gen_x)
        g_loss = bce_loss(pred_gen, real_labels)
        g_loss.backward()
        opt_g.step()
    generator.eval()
    noise_full = torch.randn(n_train, z_dim, device=device)
    X_fake_full = generator(noise_full).detach()  # shape: (N, 8)

    half_n = n_train // 2
    real_idx = perm[:half_n]
    X_real_batch = X_train[real_idx]
    y_real_batch = y_train[real_idx]
    X_fake_batch = X_fake_full[:(n_train - half_n)]
    model1.train()
    model2.train()

    y_hat = model1(X_real_batch)
    loss_m1_mse = mse_loss(y_hat, y_real_batch)

    # 2) LIME explanations on X_real_batch
    X_real_batch_np = X_real_batch.cpu().numpy()
    lime_expls = lime_explanations(
        model=model1,
        lime_explainer=lime_explainer,
        X_batch_np=X_real_batch_np,
        feature_count=len(features)
    )

    # 3) Prepare input for Model2 => [X, y_hat, LIME]
    y_hat_np = y_hat.detach().cpu().numpy().reshape(-1, 1)
    input_model2_np = np.concatenate([X_real_batch_np, y_hat_np, lime_expls], axis=1)
    input_model2 = torch.tensor(input_model2_np, dtype=torch.float32, device=device)

    # 4) Model2 forward
    y_eval_pred = model2(input_model2)
    loss_m2 = mse_loss(y_eval_pred, y_real_batch)

    # 5) Feedback loss
    feedback_loss = loss_m2
    loss_m1_total = loss_m1_mse + alpha * feedback_loss

    # 6) Optimize
    opt_m1.zero_grad()
    opt_m2.zero_grad()

    loss_m1_total.backward(retain_graph=True)

    opt_m2.zero_grad()
    loss_m2.backward()

    opt_m1.step()
    opt_m2.step()
    print(f"\nEpoch [{epoch+1}/{epochs}] => "
              f"GAN step [Discriminator Loss: {d_loss.item():.4f}, Generator Loss: {g_loss.item():.4f}] "
              f"| Model1 MSE: {loss_m1_mse.item():.4f}, Model2 MSE: {loss_m2.item():.4f}, "
              f"Total M1 Loss: {loss_m1_total.item():.4f}")



Epoch [1/2] => GAN step [Discriminator Loss: 1.3575, Generator Loss: 0.6944] | Model1 MSE: 0.0806, Model2 MSE: 0.5462, Total M1 Loss: 0.3537

Epoch [2/2] => GAN step [Discriminator Loss: 1.4076, Generator Loss: 0.7242] | Model1 MSE: 0.0788, Model2 MSE: 0.5333, Total M1 Loss: 0.3455


In [ ]:
model1.eval()
with torch.no_grad():
    y_hat_test = model1(X_test)
    test_mse = mse_loss(y_hat_test, y_test).item()
    test_rmse = np.sqrt(test_mse)
    test_mae = mean_absolute_error(y_test.cpu().numpy(), y_hat_test.cpu().numpy())
    r2 = -r2_score(y_test.cpu().numpy(), y_hat_test.cpu().numpy())

print("\n--- Final Evaluation of Model 1 on Test ---")
print(f"MSE:  {test_mse:.4f}")
print(f"RMSE: {test_rmse:.4f}")
print(f"MAE:  {test_mae:.4f}")
print(f"R^2:  {r2:.4f}")

# Accuracy by error
y_max = torch.max(y).item()
y_min = torch.min(y).item()
alpha_range = y_max - y_min

mae_accuracy = (1.0 - test_mae / alpha_range) * 100.0
rmse_accuracy = (1.0 - test_rmse / alpha_range) * 100.0
mse_accuracy = (1.0 - test_mse / alpha_range) * 100.0

print("\n--- Accuracy by Error Metrics ---")
print(f"MSE:  {test_mse:.4f},  Accuracy: {mse_accuracy:.2f}%")
print(f"RMSE: {test_rmse:.4f}, Accuracy: {rmse_accuracy:.2f}%")
print(f"MAE:  {test_mae:.4f},  Accuracy: {mae_accuracy:.2f}%")


--- Final Evaluation of Model 1 on Test ---
MSE:  0.0764
RMSE: 0.2764
MAE:  0.2206
R^2:  0.6248

--- Accuracy by Error Metrics ---
MSE:  0.0764,  Accuracy: 92.36%
RMSE: 0.2764, Accuracy: 72.36%
MAE:  0.2206,  Accuracy: 77.94%
